In [51]:

from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [5]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant",
    instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not",
    tools=functions,
    model="gpt-4o-mini-2024-07-18"
)

Assistant(id='asst_sKt1h6EHSAfC92K6RqMw8WAF', created_at=1726150632, description=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not', metadata={}, model='gpt-4o-mini-2024-07-18', name='Investor Assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type': 'object', 'properties': {'ticker': {'type': 'string', 'description': 'Ticker symbol of the company'}}, 'required': ['ticker']}, strict=False), type='function'), FunctionTool(function=Fu

In [8]:
assistant_id = "asst_sKt1h6EHSAfC92K6RqMw8WAF"

In [46]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the AT&T stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_ebdz4iPCa2W54MCWnyKJslcQ', created_at=1726156297, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [64]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [47]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_NEPn2W73qHDvd2umrO9fwQud', assistant_id='asst_sKt1h6EHSAfC92K6RqMw8WAF', cancelled_at=None, completed_at=None, created_at=1726156301, expires_at=1726156901, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_R15vo3uD0hiHgjvbXYMFHP8M', function=Function(arguments='{"company_name":"AT&T"}', name='get_ticker'), type='function')]), type='submit_tool_outputs'), response_format=ResponseFormatText(type='text'), started_at=1726156301, status='requires_action', thread_id='thread_ebdz4iPCa2W54MCWnyKJslcQ', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinitio

In [60]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

requires_action


In [63]:
get_messages(thread.id)

user: I want to know if the AT&T stock is a good buy
assistant: To determine if AT&T Inc. (Ticker: T) is a good buy, let's analyze its financial performance, balance sheet, and recent stock performance.

### Financial Performance (Income Statement)
1. **Total Revenue**: $122.43 billion
2. **Net Income**: $14.4 billion
3. **Diluted EPS (Earnings Per Share)**: $1.97
4. **Total Expenses**: $97.77 billion
5. **Operating Income**: $23.46 billion

### Balance Sheet
1. **Total Assets**: $407.06 billion
2. **Total Liabilities**: $287.65 billion
3. **Stockholders' Equity**: $103.30 billion
4. **Total Debt**: $154.90 billion
5. **Net Debt**: $130.61 billion

### Recent Stock Performance
- **Current Price**: Approximately $20.96
- **52-week Range**: The stock has fluctuated between approximately $17.30 and $21.86.
- **Recent Trends**: The stock has shown some volatility but has been trending upwards recently.

### Key Considerations
- **Debt Levels**: AT&T has a significant amount of debt, which 

In [62]:
get_run(run.id, thread.id).status

'completed'

In [58]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "T"}
Calling function: get_balance_sheet with arg {"ticker": "T"}
Calling function: get_daily_stock_performance with arg {"ticker": "T"}


[{'output': '"{\\"1703980800000\\":{\\"Tax Effect Of Unusual Items\\":-254109000.0,\\"Tax Rate For Calcs\\":0.213,\\"Normalized EBITDA\\":46522000000.0,\\"Total Unusual Items\\":-1193000000.0,\\"Total Unusual Items Excluding Goodwill\\":-1193000000.0,\\"Net Income From Continuing Operation Net Minority Interest\\":14400000000.0,\\"Reconciled Depreciation\\":18777000000.0,\\"Reconciled Cost Of Revenue\\":50123000000.0,\\"EBITDA\\":45329000000.0,\\"EBIT\\":26552000000.0,\\"Net Interest Income\\":-6704000000.0,\\"Interest Expense\\":6704000000.0,\\"Normalized Income\\":15338891000.0,\\"Net Income From Continuing And Discontinued Operation\\":14400000000.0,\\"Total Expenses\\":97774000000.0,\\"Total Operating Income As Reported\\":23461000000.0,\\"Diluted Average Shares\\":7258000000.0,\\"Basic Average Shares\\":7181000000.0,\\"Diluted EPS\\":1.97,\\"Basic EPS\\":1.97,\\"Diluted NI Availto Com Stockholders\\":14277000000.0,\\"Average Dilution Earnings\\":85000000.0,\\"Net Income Common Sto

In [59]:
submit_tool_outputs(run.id , thread.id )

Calling function: get_income_statement with arg {"ticker": "T"}
Calling function: get_balance_sheet with arg {"ticker": "T"}
Calling function: get_daily_stock_performance with arg {"ticker": "T"}


Run(id='run_NEPn2W73qHDvd2umrO9fwQud', assistant_id='asst_sKt1h6EHSAfC92K6RqMw8WAF', cancelled_at=None, completed_at=None, created_at=1726156301, expires_at=1726156901, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='thread.run', parallel_tool_calls=True, required_action=None, response_format=ResponseFormatText(type='text'), started_at=1726156412, status='queued', thread_id='thread_ebdz4iPCa2W54MCWnyKJslcQ', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='functio